In [24]:
from constants import *
import cv2
import pandas as pd
import numpy as np
from PIL import Image

cascade_classifier = cv2.CascadeClassifier(CASC_PATH)


In [26]:
FILE_PATH = './data/fer2013.csv'
data = pd.read_csv(FILE_PATH)

In [65]:
def format_image(image):
  if len(image.shape) > 2 and image.shape[2] == 3:
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  else:
    image = cv2.imdecode(image, cv2.CV_LOAD_IMAGE_GRAYSCALE)
  gray_border = np.zeros((150, 150), np.uint8)
  gray_border[:,:] = 200
  gray_border[((150 / 2) - (SIZE_FACE/2)):((150/2)+(SIZE_FACE/2)), ((150/2)-(SIZE_FACE/2)):((150/2)+(SIZE_FACE/2))] = image
  image = gray_border

  faces = cascade_classifier.detectMultiScale(
    image,
    scaleFactor = 1.3,
    minNeighbors = 5
  )
  # None is we don't found an image
  if not len(faces) > 0:
    #print "No hay caras"
    return None
  max_area_face = faces[0]
  for face in faces:
    if face[2] * face[3] > max_area_face[2] * max_area_face[3]:
      max_area_face = face
  # Chop image to face
  face = max_area_face
  image = image[face[1]:(face[1] + face[2]), face[0]:(face[0] + face[3])]
  # Resize image to network size

  try:
    image = cv2.resize(image, (SIZE_FACE, SIZE_FACE), interpolation = cv2.INTER_CUBIC) / 255.
  except Exception:
    print("[+] Problem during resize")
    return None
  print(image.shape)
  return image

def emotion_to_vec(x):
    d = np.zeros(len(EMOTIONS))
    d[x] = 1.0
    return d

def data_to_image(data):
    #print data
    data_image = np.fromstring(str(data), dtype = np.uint8, sep = ' ').reshape((SIZE_FACE, SIZE_FACE))
    data_image = Image.fromarray(data_image).convert('RGB')
    data_image = np.array(data_image)[:, :, ::-1].copy() 
    data_image = format_image(data_image)
    return data_image

In [66]:
labels = []
images = []
index = 1
total = data.shape[0]
for index, row in data.iterrows():
    emotion = emotion_to_vec(row['emotion'])
    image = data_to_image(row['pixels'])
    if image is not None:
        labels.append(emotion)
        images.append(image)
    else:
        print("Error")
        
    index += 1
    
   
    print("Progreso: {}/{} {:.2f}%".format(index, total, index * 100.0 / total))

/home/ryan/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:8: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


(48, 48)
Progreso: 1/35887 0.00%
Error
Progreso: 2/35887 0.01%
(48, 48)
Progreso: 3/35887 0.01%
Error
Progreso: 4/35887 0.01%
(48, 48)
Progreso: 5/35887 0.01%
(48, 48)
Progreso: 6/35887 0.02%
Error
Progreso: 7/35887 0.02%
(48, 48)
Progreso: 8/35887 0.02%
(48, 48)
Progreso: 9/35887 0.03%
(48, 48)
Progreso: 10/35887 0.03%
(48, 48)
Progreso: 11/35887 0.03%
Error
Progreso: 12/35887 0.03%
Error
Progreso: 13/35887 0.04%
Error
Progreso: 14/35887 0.04%
(48, 48)
Progreso: 15/35887 0.04%
(48, 48)
Progreso: 16/35887 0.04%
Error
Progreso: 17/35887 0.05%
Error
Progreso: 18/35887 0.05%
Error
Progreso: 19/35887 0.05%
Error
Progreso: 20/35887 0.06%
Error
Progreso: 21/35887 0.06%
Error
Progreso: 22/35887 0.06%
(48, 48)
Progreso: 23/35887 0.06%
Error
Progreso: 24/35887 0.07%
Error
Progreso: 25/35887 0.07%
(48, 48)
Progreso: 26/35887 0.07%
Error
Progreso: 27/35887 0.08%
Error
Progreso: 28/35887 0.08%
Error
Progreso: 29/35887 0.08%
Error
Progreso: 30/35887 0.08%
(48, 48)
Progreso: 31/35887 0.09%
Error
Pro

In [72]:
print("Total: " + str(len(images)))
np.save('data_kike.npy', images)
np.save('labels_kike.npy', labels)

Total: 13975
